In [1]:
import os
os.chdir('/content/drive/MyDrive/MVA/KKML')

# Kernel Methods: Challenge

Julia Linhart, Roman Castagné, Louis Bouvier

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functools import partial
from scipy.spatial import distance_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV
import cvxpy as cp
import warnings
import time
from itertools import product
from numba import jit

from utils import run_model, write_csv


warnings.filterwarnings("ignore", category=DeprecationWarning)

# I) Preprocessing

In [2]:
data_folder = 'data' # 'machine-learning-with-kernel-methods-2021'

X_train_1 = pd.read_csv(f'{data_folder}/Xtr2_mat100.csv', sep = ' ', index_col=False, header=None)
y_train_1 = pd.read_csv(f'{data_folder}/Ytr2.csv')

In [3]:
y_train_1.describe()

,Id,Bound
count,2000.000000,2000.000000
mean,4999.500000,0.498500
std,577.494589,0.500123
min,4000.000000,0.000000
25%,4499.750000,0.000000
50%,4999.500000,0.000000
75%,5499.250000,1.000000
max,5999.000000,1.000000


In [4]:
y_train_1 = np.array(y_train_1)[:,1]

In [5]:
X_train_1.describe()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.010565,0.010201,0.010375,0.011587,0.011609,0.010707,0.009359,0.011957,0.009571,0.010582,...,0.007951,0.009457,0.008554,0.009283,0.008261,0.009614,0.011141,0.009777,0.008217,0.008565
std,0.012278,0.010723,0.011467,0.011453,0.012182,0.010478,0.009789,0.012444,0.013805,0.013652,...,0.009605,0.009701,0.009350,0.009741,0.012341,0.010338,0.010863,0.010402,0.009709,0.009283
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.010870,0.010870,0.010870,0.010870,0.010870,0.010870,0.010870,0.010870,0.000000,0.010870,...,0.010870,0.010870,0.010870,0.010870,0.000000,0.010870,0.010870,0.010870,0.010870,0.010870
75%,0.010870,0.021739,0.021739,0.021739,0.021739,0.021739,0.010870,0.021739,0.010870,0.021739,...,0.010870,0.010870,0.010870,0.010870,0.010870,0.010870,0.021739,0.010870,0.010870,0.010870
max,0.086957,0.065217,0.097826,0.065217,0.065217,0.054348,0.054348,0.076087,0.097826,0.184783,...,0.054348,0.065217,0.054348,0.054348,0.086957,0.065217,0.076087,0.065217,0.065217,0.043478


In [ ]:
X_train_1 = np.array(X_train_1)
X_train_1 = (X_train_1 - X_train_1.mean(axis=0))/X_train_1.std(axis=0)

# II) First linear models of the mat100 input

## A) Logistic regression

In [22]:
from utils import run_model

run_model('logreg')

C:\Users\roman\Documents\Academic\2020_MVA\S2_KernelMethods\KM_challenge\functions.py:103: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  inv_hess, _, _, _ = np.linalg.lstsq(hess, np.eye(hess.shape[0]))
C:\Users\roman\Documents\Academic\2020_MVA\S2_KernelMethods\KM_challenge\functions.py:111: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  inv_hess, _, _, _ = np.linalg.lstsq(hess, np.eye(hess.shape[0]))


Accuracy on train set 0: 0.62
Accuracy on test set 0 : 0.56
Accuracy on train set 1: 0.60
Accuracy on test set 1 : 0.59
Accuracy on train set 2: 0.70
Accuracy on test set 2 : 0.66


In [ ]:
from utils import write_csv

ids = np.arange(all_y_eval.shape[0])
filename = "results/submission_log_reg.csv"

# write_csv(ids, all_y_eval, filename)

## B) Ridge regression

In [10]:
run_model('rr')

Accuracy on train set 0: 0.65
Accuracy on test set 0 : 0.60
Accuracy on train set 1: 0.64
Accuracy on test set 1 : 0.57
Accuracy on train set 2: 0.73
Accuracy on test set 2 : 0.69


# III) Kernel baselines 

## A) Gaussian Kernel

### a) Kernel Ridge Regression

In [5]:
## run kernel ridge regression with gaussian kernel
run_model('krr', kernel='gaussian', prop_test=0.2, use_grid_search=True)

{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.5789473684210527}
Accuracy on train set 0: 1.00
Accuracy on test set 0 : 0.57
{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.6578947368421053}
Accuracy on train set 1: 1.00
Accuracy on test set 1 : 0.59
{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.5}
Accuracy on train set 2: 1.00
Accuracy on test set 2 : 0.67


### b) Kernel SVM

In [9]:
## run kernel SVM with gaussian kernel
run_model('ksvm', kernel='gaussian', prop_test=0.2)

{'kernel': 'gaussian', 'lamb': 1e-07, 'sigma': 100.0}
Accuracy on train set 0: 0.99
Accuracy on test set 0 : 0.62
{'kernel': 'gaussian', 'lamb': 1e-10, 'sigma': 1.0}
Accuracy on train set 1: 1.00
Accuracy on test set 1 : 0.60
{'kernel': 'gaussian', 'lamb': 1e-07, 'sigma': 100.0}
Accuracy on train set 2: 0.99
Accuracy on test set 2 : 0.72


## B) Spectrum kernel

In [3]:
from kernels import Spectrum_kernel

In [38]:
# Example when using a precomputed kernel
K = []
for name in [0, 1, 2]:
    X    = np.array(pd.read_csv(f'{data_folder}/Xtr{name}.csv')['seq'])
    X_ev = np.array(pd.read_csv(f'{data_folder}/Xte{name}.csv')['seq'])
    
    t0 = time.time()
    K_tr = Spectrum_kernel(X, X, k=6)
    print(f"Time to compute train kernel : {time.time() - t0}")
    K_te = Spectrum_kernel(X, X_ev, k=6)
    
    K.append({"train": K_tr, "eval": K_te})

Time to compute train kernel : 0.7150859832763672
Time to compute train kernel : 0.6163516044616699
Time to compute train kernel : 0.6303155422210693


In [39]:
## run kernel SVM with gaussian kernel
run_model('ksvm', kernel='spectrum', K=K, sequence=True, prop_test=0.2)

KeyboardInterrupt: 

### a) Kernel SVM


## C) Substring kernel

In [13]:
from kernels import substring_similarity, substring_kernel

In [12]:
# Run similarity between two strings
t0 = time.time()
# k = substring_similarity("ATGCATGATGCATG", "ATGCATCATGATGT", 3, 1.)
# k = substring_similarity("ATGC", "ATGC", 3, 0.7)
k = substring_similarity("cat", "cat", 1, 0.7)
k_expected = 2 * 0.7 ** 4 + 0.7 ** 6
print(f"Time to compute : {time.time() - t0:.4f}s")
print(f"Value : {k}")
print(f"Expected value : {k_expected}")

Time to compute : 2.0617s
Value : 0.9799999999999999
Expected value : 0.5978489999999999


In [14]:
# Run kernel computation between N strings
X = pd.read_csv(f'{data_folder}/Xtr0.csv', sep = ',').to_numpy()
X = X[:100,1]
t0 = time.time()
K = substring_kernel(X, X, k=5, lambd=0.7)
print(f"Time to compute K : {time.time() - t0:.2f}s")
# print(K)

Time to compute K : 76.18s


## D) Mismatch kernel

In [24]:
from kernels import Mismatch_kernel

In [46]:
# Example when using a precomputed kernel
K = []
for name in [0, 1, 2]:
    X    = np.array(pd.read_csv(f'{data_folder}/Xtr{name}.csv')['seq'])
    X_ev = np.array(pd.read_csv(f'{data_folder}/Xte{name}.csv')['seq'])
    
    t0 = time.time()
    K_tr = Mismatch_kernel(X, X, k=7, m=3)
    print(f"Time to compute train kernel : {time.time() - t0}")
    K_te = Mismatch_kernel(X, X_ev, k=7, m=3)
    
    K.append({"train": K_tr, "eval": K_te})

Time to compute train kernel : 11390.832644939423
Time to compute train kernel : 289.070552110672
Time to compute train kernel : 269.14104986190796


In [56]:
y_eval = run_model('ksvm', kernel='mismatch', K=K, sequence=True, prop_test=0.001)

Accuracy on train set 0: 0.87
Accuracy on test set 0 : 0.50
Accuracy on train set 1: 0.88
Accuracy on test set 1 : 0.50
Accuracy on train set 2: 0.91
Accuracy on test set 2 : 0.00


In [57]:
ids = np.arange(y_eval.shape[0])
filename = "results/submission_mismatch_7_3.csv"
write_csv(ids, y_eval, filename)